Predicting patient mortality within 365 days of discharge

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt
from datetime import datetime
import shap
import tqdm

# Load and clean data

In [ ]:
# Load data
patients = pd.read_csv("PATIENTS.csv.gz")
admissions = pd.read_csv("ADMISSIONS.csv.gz")

In [ ]:
# Remove patients that died in the hospital and newborns
admissions = admissions.query("HOSPITAL_EXPIRE_FLAG != 1")
admissions = admissions.query("ADMISSION_TYPE != 'NEWBORN'")
patients = patients[patients["SUBJECT_ID"].isin(admissions["SUBJECT_ID"])]

In [ ]:
# Get latest for patients with multiple admissions
admissions = admissions.sort_values(["SUBJECT_ID", "DISCHTIME"], ascending=False).drop_duplicates(["SUBJECT_ID"], keep="first")

In [ ]:
# Merge patients and admissions
merge_df = patients.drop(columns="ROW_ID").merge(admissions.drop(columns="ROW_ID"), on="SUBJECT_ID", how="inner")

In [ ]:
# Get target variable: whether patient died within __ days after discharge
N_DAYS = 365

def get_target(disch, dod):
    # If patient hasn't died, then target is 0
    if type(dod) != str:
        return 0
    # Otherwise, check how long after discharge they died
    else:
        disch_date = datetime.strptime(disch, "%Y-%m-%d %H:%M:%S")
        dod_date = datetime.strptime(dod, "%Y-%m-%d %H:%M:%S")
        if (dod_date - disch_date).days <= N_DAYS:
            return 1
        else:
            return 0

merge_df.loc[:, "target"] = merge_df.apply(lambda row: get_target(row["DISCHTIME"], row["DOD"]), axis=1)

In [ ]:
merge_df["target"].sum()

In [ ]:
# Add age at discharge time
def get_age(dob, disch):
    dob_date = datetime.strptime(dob, "%Y-%m-%d %H:%M:%S")
    disch_date = datetime.strptime(disch, "%Y-%m-%d %H:%M:%S")
    age = (disch_date - dob_date).days // 365
    return age

merge_df.loc[:, "age"] = merge_df.apply(lambda row: get_age(row["DOB"], row["DISCHTIME"]), axis=1)

In [ ]:
# Drop invalid ages and filter to adults only
merge_df = merge_df.query("age < 200")
merge_df = merge_df.query("age >= 18")

In [ ]:
# Apply a little bit of feature engineering
merge_df.loc[:, "LANGUAGE"] = np.where(merge_df["LANGUAGE"] == "ENGL", "ENGLISH", "NOT ENGLISH")
merge_df.loc[:, "ETHNICITY"] = merge_df["ETHNICITY"].str.split(' - ').str[0]

In [ ]:
# Convert age to between 0 and 1, since all of the other data is binary
merge_df["age"] = merge_df["age"] / merge_df["age"].max()

In [ ]:
# Select feature set
FEATURES = [
    "DISCHARGE_LOCATION", "ADMISSION_TYPE", "ADMISSION_LOCATION", "INSURANCE", "LANGUAGE", "RELIGION", "MARITAL_STATUS", "ETHNICITY", "age"
]

In [ ]:
# To reduce the impact of rare categories, group rare categories as "Other"
MIN_PATIENTS = 20
for feat in FEATURES:
    if feat != "age":
        # Convert to lowercase
        merge_df[feat] = merge_df[feat].str.lower()
        
        # Get counts by current category label
        category_counts = merge_df[feat].value_counts()
        
        # Identify categories that appear at least MIN_PATIENTS times
        categories_to_keep = category_counts[category_counts >= MIN_PATIENTS].index
        
        # Step 3: Replace categories that appear 10 times or fewer with "Other"
        merge_df[feat] = merge_df[feat].apply(lambda x: x if x in categories_to_keep else 'other')

# Add procedures

In [ ]:
procedures = pd.read_csv("PROCEDURES_ICD.csv.gz")

In [ ]:
# Filter by admission (should give one admission per patient)
procedures = procedures[procedures["HADM_ID"].isin(admissions["HADM_ID"])]

In [ ]:
# Define mapping of procedure codes to higher level categories
# (based on https://www.findacode.com/code-set.php?set=ICD9V3)
proc_descriptions = [
    "OTHER",
    "NERVOUS",
    "ENDOCRINE",
    "EYE",
    "OTHER_DIAGNOSTIC_THERAPEUTIC",
    "EAR",
    "NOSE_MOUTH_PHARYNX",
    "RESPIRATORY",
    "CARDIOVASCULAR",
    "HEMIC_LYMPHATIC",
    "DIGESTIVE",
    "URINARY",
    "MALE_GENITAL",
    "FEMALE_GENITAL",
    "OBSTETRICAL",
    "MUSCULOSKELETAL",
    "INTEGUMENTARY",
    "DIAGNOSTIC_THERAPEUTIC"
]

proc_ranges = [
    ("00", "00"),
    ("01", "05"),
    ("06", "07"),
    ("08", "16"),
    ("17", "17"),
    ("18", "20"),
    ("21", "29"),
    ("30", "34"),
    ("35", "39"),
    ("40", "41"),
    ("42", "54"),
    ("55", "59"),
    ("60", "64"),
    ("65", "71"),
    ("72", "75"),
    ("76", "84"),
    ("85", "86"),
    ("87", "99")
]

proc_map = {}
for i in range(0, len(proc_descriptions)):
    start = proc_ranges[i][0]
    end = proc_ranges[i][1]
    for j in range(int(start), int(end) + 1):
        proc_map[f"{j:02}"] = proc_descriptions[i].lower()

In [ ]:
# Define function to carry out mapping of procedures to higher level categories
def get_proc_category(code):
    # Get first two digits
    cat_num = str(code)[0:2]

    # Look up in mapping
    cat = proc_map[cat_num]
    return cat

In [ ]:
# Add procedure category to dataframe
procedures["procedure_type"] = procedures.apply(lambda row: get_proc_category(row["ICD9_CODE"]), axis=1)

In [ ]:
# Convert to binary, with procedure categories as columns
procedures_agg = pd.get_dummies(procedures[["SUBJECT_ID", "procedure_type"]].drop_duplicates(), ["procedure_type"], dtype=int)
procedures_agg = procedures_agg.groupby("SUBJECT_ID").max().reset_index()

In [ ]:
# Join with rest of data
merge_df = merge_df.merge(procedures_agg, on="SUBJECT_ID", how="left")

In [ ]:
# Fill with 0s for patients with no procedures
procedure_cols = [x for x in procedures_agg.columns if x != "SUBJECT_ID"]
merge_df.loc[:, procedure_cols] = merge_df.loc[:, procedure_cols].fillna(0)

# Train and evaluate model: logistic regression

In [ ]:
# One-hot encode the feature set
X = pd.get_dummies(merge_df[FEATURES + procedure_cols], columns=FEATURES[:-1], prefix=FEATURES[:-1], dummy_na=True, dtype=int)

In [ ]:
# Get the target
y = merge_df["target"]

In [ ]:
# Split into train and test sets (50/50)
np.random.seed(777)
train_perc = 0.5
train_size = round(train_perc * X.shape[0])
train_idx = np.random.choice(X.index, train_size, replace=False)
train_mask = np.where(X.index.isin(train_idx), True, False)

X_train = X[train_mask]
X_test = X[~train_mask]
y_train = y[train_mask]
y_test = y[~train_mask]

In [ ]:
# Train logistic regression model
model_lr = LogisticRegression()
model_lr.fit(X_train, y_train)

In [ ]:
# Define function to help plot ROC curves
def plot_roc(y_prob, y_actual):
    # Calculate ROC curve
    fpr, tpr, thresholds = roc_curve(y_actual, y_prob)
    
    # Calculate AUC
    auc = roc_auc_score(y_actual, y_prob)
    
    # Plot ROC curve
    plt.figure()
    plt.plot(fpr, tpr, color="blue", lw=2, label=f"ROC curve (area = {auc:.4f})")
    plt.plot([0, 1], [0, 1], color="gray", lw=2, linestyle="--")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
# Plot ROC curve for train set
plot_roc(model_lr.predict_proba(X_train)[:, 1], y_train)

In [ ]:
# Plot ROC curve for test set
plot_roc(model_lr.predict_proba(X_test)[:, 1], y_test)

In [ ]:
# Get SHAP feature importance
explainer = shap.Explainer(model_lr, X_test)

# Calculate SHAP values
shap_values = explainer(X_test)

# Summarize the feature importances (top 10)
shap.summary_plot(shap_values, X_test, plot_type="bar", max_display=10)

# Train and evaluate model: random forest

In [ ]:
# Train random forest model
model_rf = RandomForestClassifier(min_samples_split=50, max_features=2)
model_rf.fit(X_train, y_train)

In [ ]:
# Plot ROC curve for train set
plot_roc(model_rf.predict_proba(X_train)[:, 1], y_train)

In [ ]:
# Plot ROC curve for test set
plot_roc(model_rf.predict_proba(X_test)[:, 1], y_test)

In [ ]:
# Get feature importance
# NOTE: ran into issues using SHAP for this

# Create df for coefficients
importance_df = pd.DataFrame({
    "feature": X.columns,
    "importance": model_rf.feature_importances_
    
})

# # Sort by abs val of coefficients
# importance_df["abs_coef"] = importance_df["coef"].abs()
# importance_df = importance_df.sort_values(by="abs_coef", ascending=False)

importance_df.sort_values("importance", ascending=False).head(10)

# Train and evaluate model: ensemble

In [ ]:
# Train simple model to predict target based on predictions of existing models
# Train logistic regression model
scores_train = pd.DataFrame({
    "lr_score": model_lr.predict_proba(X_train)[:, 1],
    "rf_score": model_rf.predict_proba(X_train)[:, 1]
})
# NOTE: using stronger regularization here
model_ensemble = LogisticRegression(C=0.001)
model_ensemble.fit(scores_train, y_train)

In [ ]:
# Plot ROC curve for train set
plot_roc(model_ensemble.predict_proba(scores_train)[:, 1], y_train)

In [ ]:
# Plot ROC curve for test set
scores_test = pd.DataFrame({
    "lr_score": model_lr.predict_proba(X_test)[:, 1],
    "rf_score": model_rf.predict_proba(X_test)[:, 1]
})
plot_roc(model_ensemble.predict_proba(scores_test)[:, 1], y_test)

In [ ]:
# Compare with simple ensemble approach - sum of scores
plot_roc(model_lr.predict_proba(X_test)[:, 1] + model_rf.predict_proba(X_test)[:, 1], y_test)